In [1]:
import numpy as np
import matplotlib.pyplot as plt
import progressbar

In [2]:
file_path = "Data"
files = [
    "docword.kos.txt",
    "docword.nips.txt",
    "docword.enron.txt",
    ]
max_k = 15
k_iters = 3

In [3]:
def jac_dist(v1, v2):    
    sum = v1 + v2
    union = np.count_nonzero(sum)
    intersection = np.sum(sum > 1)
    
    if union == 0:
        return 1
    
    jacc_dist = 1 - (intersection / union)
    return jacc_dist

In [4]:
def calc_inertia(data,
                 cluster_labels,
                 centroids):
    inertia = 0
    
    for i in range(len(data)):
        inertia += jac_dist(
            data[i],
            centroids[cluster_labels[i]]
            )
    
    return inertia

In [5]:
def custom_kmeans(data:list,
                  k:int,
                  dim:int,
                  max_iter=30,
                  past_centroids:list=None,
                  past_labels:list=None):
    """
    Custom K-Means implementation with a Jaccard Similarity Measure.

    Args:
        data: 
        k: The desired number of clusters.
        max_iter: Maximum number of iterations (default: 30).

    Returns:
        A tuple containing:
            cluster_labels: An array of cluster labels for each data point.
            centroids: The final centroids (cluster centers) after convergence.
            intertia: The inertia for the final centroids and clusters
    """    
    if k == 2:
        # Initialize the first centroid randomly
        rnd = np.random.choice(a=len(data),
                                size=1,
                                replace=False)
        centroids = []
        centroids.append(data[rnd[0]])
        
        cluster_labels = [0] * len(data)
    else:
        centroids = past_centroids
        cluster_labels = past_labels
        
    furthest_pt = data[0]
    max_dist = 0
    
    # Add next centroid as the one that is furthest from first centroid
    for i in range(len(data)):
        pt = data[i]
        c = centroids[cluster_labels[i]]
        
        if jac_dist(c, pt) > max_dist:
            furthest_pt = pt
            max_dist = jac_dist(c, pt)
    
    centroids.append(furthest_pt)
        
    # Re-assign cluster labels
    for i in range(len(data)):
        pt = data[i]
        c = centroids[cluster_labels[i]]
        new_c = centroids[-1]
        
        if jac_dist(c, pt) > jac_dist(new_c, pt):
            cluster_labels[i] = len(centroids) - 1

    # Iterate for max_iter or until convergence
    cluster_labels = []
    mean_calc = [[0, np.zeros(dim)]] * len(centroids)
    for _ in range(max_iter):
        old_centroids = centroids.copy()

        # Assign data points to closest centroids
        for i in range(len(data)):
            new_cluster = 0
            for j in range(len(centroids)):
                if jac_dist(data[i], centroids[new_cluster]) \
                    > jac_dist(data[i], centroids[j]):
                    new_cluster = j
            
            cluster_labels.append(new_cluster)
            mean_calc[new_cluster] = [mean_calc[new_cluster][0] + 1,
                                      mean_calc[new_cluster][1] + data[i]]
                
        # Update centroids (mean of assigned points)
        centroids = []
        for cl in mean_calc:
            if cl[0] == 0:
                centroids.append(cl[1])
            else:    
                centroids.append(cl[1] / cl[0])

        # Check for convergence
        if np.all(np.array([jac_dist(c, oc)
                            for c in centroids
                            for oc in old_centroids]) < 1e-3):
            break
    
    # Update clustering after centroids have been updated
    for i in range(len(data)):
        new_cluster = 0
        
        for j in range(len(centroids)):
            if jac_dist(data[i], centroids[new_cluster]) \
                > jac_dist(data[i], centroids[j]):
                new_cluster = j
        
        cluster_labels.append(new_cluster)
    
    # Calculate Inertia            
    inertia = calc_inertia(
                data=data,
                cluster_labels=cluster_labels,
                centroids=centroids
                )

    return cluster_labels, centroids, inertia

In [6]:
def kmeans_pipeline(file:str,
                    max_k:int,
                    k_iters:int):
    
    source = open(f'{file_path}/{file}', 'r')
    D = int(next(source).strip())
    W = int(next(source).strip())
    NNZ = int(next(source).strip())
    data = np.zeros((D, W))

    tmp = None
    for line in source:
        d, w, _ = list(map(int, line.strip().split()))
        data[d - 1][w - 1] = 1

    # Sanity Check
    read_words = np.sum(data).item()
    if read_words != NNZ:
        return "Failure: Data Read Improperly"
    
    file_name = file.split('.')[1].upper()
    
    widgets = [f'Clustering on {file_name}: ', progressbar.Percentage(), ' | ',
            progressbar.Timer(), ' | (', progressbar.ETA(), ') ']
    bar = progressbar.ProgressBar(
        maxval=(max_k - 1),
        widgets=widgets)\
            .start()
    
    k_inertia = []
    past_centroids = None
    past_labels = None
    for k in range(2, max_k + 1):
        min_inertia = 10 ** 6
        
        if k == 2:
            for _ in range(k_iters):
                cluster_labels, centroids, inertia = \
                    custom_kmeans(
                        data=data,
                        k=k,
                        dim=W,
                        past_centroids=past_centroids,
                        past_labels=past_labels
                        )

                if min_inertia > inertia:
                    min_inertia = inertia
                    min_centroids = centroids
                    min_labels = cluster_labels
        else:
            cluster_labels, centroids, inertia = \
                custom_kmeans(
                    data=data,
                    k=k,
                    dim=W,
                    past_centroids=past_centroids,
                    past_labels=past_labels
                    )

            min_inertia = inertia
            min_centroids = centroids
            min_labels = cluster_labels
        
        past_centroids = min_centroids
        past_labels = min_labels
        k_inertia.append(min_inertia)
        bar.update(k - 2 + 1)
    
    print(f'Sparsity of Matrix on {file_name}: {NNZ / (D * W) * 100:.3f} %')
    print()
    
    return k_inertia            

In [7]:
logs = []
for file in files:
    k_inertia = kmeans_pipeline(
                    file=file,
                    max_k=max_k,
                    k_iters=k_iters)
    
    file_name = file.split('.')[1].upper()
    
    plt.figure(figsize=(10, 4))
    plt.plot(list(range(2, max_k + 1)),
             k_inertia,
             marker='o', 
             linestyle='-')
    plt.title(f'Clustering on {file_name}')
    plt.xlabel('K')
    plt.ylabel('Inertia (Jaccard Similarity)')
    plt.ylim([max(min(k_inertia) - 0.01 * max(k_inertia), 0), 1.01 * max(k_inertia)])
    plt.savefig(f'Output/KM++ - MR/{file_name}.png')
    plt.close()
    
    logs.append((file_name, k_inertia))

Sparsity of Matrix on KOS: 1.491 %



Sparsity of Matrix on NIPS: 4.006 %



KeyboardInterrupt: 